<a href="https://colab.research.google.com/github/Teasotea/DialogSystem/blob/main/ConversationalAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!pip install transformers

In [39]:
!pip install nltk

In [40]:
import numpy as np
import pandas as pd
import time
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import nltk
from sklearn.svm import OneClassSVM
import re

# Part I: Classification

## Greeting Classification

In [41]:
training_data_greet, training_data_bye = [], []
greetings =list(dict.fromkeys(['hi', "hola", 'hey', 'hello','morning', 'evening', 'good day', 'good morning', 'greetings', 'howdy', 'welcome', 'bonjour',
             'buenas noches', 'buenos dias', 'salutation', 'salut', 'hail', 'salaam', 'aloha', 'good wishes',
             'aloha', 'yoo-hoo', 'yawp', 'It’s always a pleasure to see you.', 'oh', 'ave', 'yo', 'hi there', 
             "hi", "hello", "hey", "helloo", "hellooo", "g morining", "gmorning", "good morning", "morning", "good day", "good afternoon", "good evening", "greetings", "greeting", 
             "good to see you", "its good seeing you", "g’day", "howdy"]))
goodbyes = list(dict.fromkeys(['bye', 'exit', 'quit', 'See you later', 'I must be going', 'I’m off', 
                               'goodbye', 'Goodnight','I gotta take off', 'I’ve got to get going', 
                               'See ya!', 'Catch ya later', 'Ciao', 'Adios' ]))

for i in greetings:
  training_data_greet.append({"class":"greeting", "sentence":i})
for i in goodbyes:
  training_data_bye.append({'class':'goodbyes', 'sentence': i})
  
greet_df = pd.DataFrame(training_data_greet)
bye_df = pd.DataFrame(training_data_bye)

bye_df

,class,sentence
0,goodbyes,bye
1,goodbyes,exit
2,goodbyes,quit
3,goodbyes,See you later
4,goodbyes,I must be going
5,goodbyes,I’m off
6,goodbyes,goodbye
7,goodbyes,Goodnight
8,goodbyes,I gotta take off
9,goodbyes,I’ve got to get going


In [42]:
greetings_test = ['hi, there!','psst, can you tell me, which algorithm would work best if I have imbalanced dataset?','What\'s your favourite music?','Hello? Somebody is here?', 'people often say welcome to guests','How many answers can you provide on this topic?',"hola, hola!", 'tell me about yourself','are you a bot','which movie would ypu suggest to watch this evening?', 'which laptop is the best for Data Science?', 'What about language generation task?','hey, bot:)', 'water']
testy_greet = np.array([ 1, -1, -1, 1, -1, -1, 1, -1,  -1, -1, -1,  -1,  1, -1])

bye_test = ['bye, bot:)', 'OK, then see you tomorrow', 'what does \'exit\' mean?', 'I\'ll take off my hat behind you, bot', 'Let\'s quit this conversation', 'How to translate \'adios\'?', 'Catch me, if you can!', 'Too boring( OK, see tou late then']
testy_bye = np.array([ 1, 1, -1, -1, 1, -1, -1, 1])

### One-Class Classification with SVM

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score

In [45]:
num_words = 1000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

In [63]:
tokenizer_greet = Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer_greet.fit_on_texts(greetings)
word_index_greet = tokenizer_greet.word_index
train_sequences_greet = tokenizer_greet.texts_to_sequences(greetings)

# Get max training sequence length
maxlen_greet = max([len(x) for x in train_sequences_greet])

# Pad the training sequences
train_padded_greet = pad_sequences(train_sequences_greet, padding=pad_type, truncating=trunc_type, maxlen=maxlen_greet)

# Output the results of our work
print("Word index:\n", word_index_greet)
print("\nTraining sequences:\n", train_sequences_greet)
# print("\nPadded training sequences:\n", train_padded)
print("\nPadded training shape:", train_padded_greet.shape)
print("Training sequences data type:", type(train_sequences_greet))
print("Padded Training sequences data type:", type(train_padded_greet))

test_sequences_greet = tokenizer_greet.texts_to_sequences(greetings_test)
test_padded_greet = pad_sequences(test_sequences_greet, padding=pad_type, truncating=trunc_type, maxlen=maxlen_greet)

print("Testing sequences:\n", test_sequences_greet)
print("\nPadded testing sequences:\n", test_padded_greet)
print("\nPadded testing shape:",test_padded_greet.shape)

for x, y in zip(greetings_test, test_padded_greet):
  print('{} -> {}'.format(x, y))

Word index:
 {'<UNK>': 1, 'good': 2, 'you': 3, 'hi': 4, 'morning': 5, 'evening': 6, 'to': 7, 'see': 8, 'hola': 9, 'hey': 10, 'hello': 11, 'day': 12, 'greetings': 13, 'howdy': 14, 'welcome': 15, 'bonjour': 16, 'buenas': 17, 'noches': 18, 'buenos': 19, 'dias': 20, 'salutation': 21, 'salut': 22, 'hail': 23, 'salaam': 24, 'aloha': 25, 'wishes': 26, 'yoo': 27, 'hoo': 28, 'yawp': 29, 'it’s': 30, 'always': 31, 'a': 32, 'pleasure': 33, 'oh': 34, 'ave': 35, 'yo': 36, 'there': 37, 'helloo': 38, 'hellooo': 39, 'g': 40, 'morining': 41, 'gmorning': 42, 'afternoon': 43, 'greeting': 44, 'its': 45, 'seeing': 46, 'g’day': 47}

Training sequences:
 [[4], [9], [10], [11], [5], [6], [2, 12], [2, 5], [13], [14], [15], [16], [17, 18], [19, 20], [21], [22], [23], [24], [25], [2, 26], [27, 28], [29], [30, 31, 32, 33, 7, 8, 3], [34], [35], [36], [4, 37], [38], [39], [40, 41], [42], [2, 43], [2, 6], [44], [2, 7, 8, 3], [45, 2, 46, 3], [47]]

Padded training shape: (37, 7)
Training sequences data type: <class 'l

In [56]:
# tokenizer_bye= Tokenizer(num_words=num_words, oov_token=oov_token)
# tokenizer_bye.fit_on_texts(goodbyes)
# word_index = tokenizer_bye.word_index
# train_sequences_bye = tokenizer_bye.texts_to_sequences(goodbyes)

# # Get max training sequence length
# maxlen_bye = max([len(x) for x in train_sequences_bye])

# # Pad the training sequences
# train_padded_bye = pad_sequences(train_sequences_bye, padding=pad_type, truncating=trunc_type, maxlen=maxlen_bye)

# # Output the results of our work
# print("Word index:\n", word_index)
# print("\nTraining sequences:\n", train_sequences_bye)
# # print("\nPadded training sequences:\n", train_padded)
# print("\nPadded training shape:", train_padded_bye.shape)
# print("Training sequences data type:", type(train_sequences_bye))
# print("Padded Training sequences data type:", type(train_padded_bye))

# test_sequences_bye = tokenizer_greet.texts_to_sequences(bye_test)
# test_padded_bye = pad_sequences(test_sequences_bye, padding=pad_type, truncating=trunc_type, maxlen=maxlen_bye)

# print("Testing sequences:\n", test_sequences_bye)
# print("\nPadded testing sequences:\n", test_padded_bye)
# print("\nPadded testing shape:",test_padded_bye.shape)

# for x, y in zip(bye_test, test_padded_bye):
#   print('{} -> {}'.format(x, y))

Word index:
 {'<UNK>': 1, 'see': 2, 'later': 3, 'i': 4, 'going': 5, 'off': 6, 'ya': 7, 'bye': 8, 'exit': 9, 'quit': 10, 'you': 11, 'must': 12, 'be': 13, 'i’m': 14, 'goodbye': 15, 'goodnight': 16, 'gotta': 17, 'take': 18, 'i’ve': 19, 'got': 20, 'to': 21, 'get': 22, 'catch': 23, 'ciao': 24, 'adios': 25}

Training sequences:
 [[8], [9], [10], [2, 11, 3], [4, 12, 13, 5], [14, 6], [15], [16], [4, 17, 18, 6], [19, 20, 21, 22, 5], [2, 7], [23, 7, 3], [24], [25]]

Padded training shape: (14, 5)
Training sequences data type: <class 'list'>
Padded Training sequences data type: <class 'numpy.ndarray'>
Testing sequences:
 [[1, 1], [1, 1, 8, 3, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 3, 1], [1, 1, 1, 1], [1, 7, 1, 1], [1, 1, 1, 3, 1], [1, 1, 1, 8, 1, 1, 1]]

Padded testing sequences:
 [[1 1 0 0 0]
 [1 1 8 3 1]
 [1 1 1 1 0]
 [1 1 1 1 1]
 [1 1 1 1 0]
 [1 7 1 1 0]
 [1 1 1 3 1]
 [1 1 1 8 1]]

Padded testing shape: (8, 5)
bye, bot:) -> [1 1 0 0 0]
OK, then see you tomorrow -> [1 1 8 3 1]
what does 'exit' m

### Greetings

In [64]:
model = OneClassSVM(gamma='scale', nu=0.01)
model.fit(train_padded_greet)

OneClassSVM(nu=0.01)

In [65]:
#+1 for normal examples, so-called inliers, and a -1 for outliers.
yhat_greet = model.predict(test_padded_greet)

In [66]:
yhat_greet

array([ 1, -1, -1,  1, -1, -1,  1, -1, -1, -1, -1, -1,  1, -1])

In [67]:
score = f1_score(testy_greet, yhat_greet, pos_label=-1)
print('F1 Score: %.3f' % score)

F1 Score: 1.000


In [75]:
def classify_greet(sent):
  l = [sent]
  test_sequences = tokenizer_greet.texts_to_sequences(l)
  test_padded= pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen_greet)
  pred_class_greet = model.predict(test_padded)
  cl = int(*pred_class_greet)
  return  cl
  
print(classify_greet('Hey! Lets talk, bot'))

1


### gooodbyes

# Part II: Question Answering

In [ ]:
!pip install datasets

In [ ]:
import datasets
import requests

# [ds for ds in datasets.list_datasets() if 'ml' in ds.lower()]

In [ ]:
qa_ds = datasets.load_dataset('squad', streaming = False)
qa_ds

In [ ]:
qa_ds['train'].description

In [ ]:
print(qa_ds['train'].dataset_size)
qa_ds['train'].features

In [ ]:
qa_ds['train'].to_pandas().head()

## Data Science Interview Dataset

In [ ]:
qa_data = pd.read_csv('https://raw.githubusercontent.com/Kizuna-Cheng/Data_Science_Interviews_NLP/main/data.csv')
qa_data.head(7)

In [ ]:
qa_data.Questions[:10].tolist()

In [ ]:
test_data = [
             'What does linear regression stands for?',
 'What is the differencebetween collinearity and multicollinearity?',
 'What are the cons of using a linear model?\n',
 'What are ridge and lasso regression?',
 'How does K-Nearest neighbor work?',
 'How to select k for k means?',
 'Why is Naive Bayes “naive”?',
 'When should I use SVM?',
'What is pruning in decision trees?',
 'What are random forests? Why is Naive Bayes better?']

## QA Baseline

In [ ]:
def getResults(questions, fn):
    def getResult(q):
        answer, score, prediction = fn(q)
        return [q, prediction, answer, score]
    return pd.DataFrame(list(map(getResult, questions)), columns=["Q", "Prediction", "A", "Score"])

In [ ]:
def getNaiveAnswer(q):
    row = qa_data.loc[qa_data['Questions'].str.contains(re.sub(r"[^\w'\s)]+", "", q),case=False)]
    if len(row) > 0:
        return row["Answers"].values[0], 1, row["Questions"].values[0]
    else: return "Sorry, I didn't get you", 0, ""
print(getNaiveAnswer('How does K-Nearest Neighbor work?'))
getResults(test_data, getNaiveAnswer)

# Part III: Natural Language Generation

In [ ]:
# checkpoint 
checkpoint = "microsoft/DialoGPT-medium"
# download and cache tokenizer
tokenizer2 = AutoTokenizer.from_pretrained(checkpoint)
# download and cache pre-trained model
modelNLG = AutoModelForCausalLM.from_pretrained(checkpoint)

# Part IV: Bot Development

In [ ]:
#change the code later, make it better
class ChatBot():
    def __init__(self):
        # once chat starts, the history will be stored for chat continuity
        self.chat_history_ids = None
        # make input ids global to use them anywhere within the object
        self.bot_input_ids = None
        # a flag to check whether to end the conversation
        self.end_chat = False
        # greet while starting
        self.welcome()
        self.is_greeting = False
        self.text_input = None
        
    def welcome(self):
        print("Initializing ChatBot ...")
        # some time to get user ready
        time.sleep(2)
        print('Type "bye" or "quit" or "exit" to end chat \n')
        # give time to read what has been printed
        time.sleep(3)

        
    def user_input(self):
        # receive input from user
        text = str(input("User    >> "))
        # end conversation if user wishes so
        if text.lower().strip() in ['bye', 'quit', 'exit']:
            # turn flag on 
            self.end_chat=True
            # a closing comment
            print('ChatBot >>  See you soon! Bye!')
            time.sleep(1)
            print('\nQuitting ChatBot ...')
        else:
            # continue chat, preprocess input text
            # encode the new user input, add the eos_token and return a tensor in Pytorch
            if classify_greet(text) == -1:
              self.text_input = text
            else: 
              self.is_greeting = True

    def bot_greet(self):
        greeting = np.random.choice([
            "Welcome, I am ChatBot, here for your kind service",
            "Hey, Great day! I am your virtual assistant",
            "Hello, it's my pleasure meeting you",
            "Hi, I am a ChatBot. Let's chat!"
        ])
        print("ChatBot >>  " + greeting)
        self.is_greeting = False

    def bot_response(self):
        self.new_user_input_ids = tokenizer2.encode(self.text_input + tokenizer2.eos_token, \
                                                       return_tensors='pt')
        # append the new user input tokens to the chat history
        # if chat has already begun
        if self.chat_history_ids is not None:
            self.bot_input_ids = torch.cat([self.chat_history_ids, self.new_user_input_ids], dim=-1) 
        else:
            # if first entry, initialize bot_input_ids
            self.bot_input_ids = self.new_user_input_ids
        
        # define the new chat_history_ids based on the preceding chats
        # generated a response while limiting the total chat history to 1000 tokens, 
        self.chat_history_ids = modelNLG.generate(self.bot_input_ids, max_length=1000, \
                                               pad_token_id=tokenizer2.eos_token_id)
            
        # last ouput tokens from bot
        response = tokenizer2.decode(self.chat_history_ids[:, self.bot_input_ids.shape[-1]:][0], \
                               skip_special_tokens=True)
        # in case, bot fails to answer
        if response == "":
            response = self.random_response()
        # print bot response
        print('ChatBot >>  '+ response)
        
    def random_response(self):
        i = -1
        response = tokenizer2.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        # iterate over history backwards to find the last token
        while response == '':
            i = i-1
            response = tokenizer2.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        # if it is a question, answer suitably
        if response.strip() == '?':
            reply = np.random.choice(["I don't know", 
                                     "I am not sure"])
        # not a question? answer suitably
        else:
            reply = np.random.choice(["Great", 
                                      "Fine. What's up?", 
                                      "Okay"
                                     ])
        return reply

In [ ]:
# build a ChatBot object
bot = ChatBot()
# start chatting
while True:
    # receive user input
    bot.user_input()
    # check whether to end chat
    if bot.end_chat:
        break
    # output bot response
    if bot.is_greeting == False:
      bot.bot_response()  
    else: bot.bot_greet()  